In [ ]:
import requests
import warnings
import os.path as osp
from pathlib import Path

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.schema.document import Document
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
llm = Ollama(model="llama2", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))
llm

Ollama(callbacks=<langchain_core.callbacks.manager.CallbackManager object at 0x79d1513a5b10>)

In [ ]:
df = pd.read_csv("../data/bbc_news.csv")
df.head()

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...


In [ ]:
documents = df.loc[:10, ["description"]].values.squeeze().tolist()
documents

['The Ukrainian president says the country will not forgive or forget those who murder its civilians.',
 'Jeremy Bowen was on the frontline in Irpin, as residents came under Russian fire while trying to flee.',
 "One of the world's biggest fertiliser firms says the conflict could deliver a shock to food supplies.",
 "The parents of the Manchester Arena bombing's youngest victim speak about their life since she died.",
 'Consumers are feeling the impact of higher energy costs as fuel prices and household bills jump.',
 'Boris Johnson is to meet the Canadian and Dutch PMs, as MPs debate new laws targeting oligarchs.',
 'The home secretary says she is "surging capacity" as about 1% of 5,500 applications are granted.',
 'TikTok suspends live streaming and new content from its platform, while Russians can no longer access Netflix.',
 'Five things you need to know about the coronavirus pandemic this Monday morning.',
 'People have been held in 53 cities, from St Petersburg in the west to Vla

In [ ]:
docs = [Document(page_content=text) for text in documents]
docs

[Document(page_content='The Ukrainian president says the country will not forgive or forget those who murder its civilians.'),
 Document(page_content='Jeremy Bowen was on the frontline in Irpin, as residents came under Russian fire while trying to flee.'),
 Document(page_content="One of the world's biggest fertiliser firms says the conflict could deliver a shock to food supplies."),
 Document(page_content="The parents of the Manchester Arena bombing's youngest victim speak about their life since she died."),
 Document(page_content='Consumers are feeling the impact of higher energy costs as fuel prices and household bills jump.'),
 Document(page_content='Boris Johnson is to meet the Canadian and Dutch PMs, as MPs debate new laws targeting oligarchs.'),
 Document(page_content='The home secretary says she is "surging capacity" as about 1% of 5,500 applications are granted.'),
 Document(page_content='TikTok suspends live streaming and new content from its platform, while Russians can no lo

In [ ]:
map_prompt_template = """
                      Write a very summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a very short and concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

In [ ]:
map_reduce_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

In [ ]:
map_reduce_outputs = map_reduce_chain({"input_documents": docs})

/home/alex/PycharmProjects/test-llms/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



In a statement made by the President of Ukraine, they vowed that their country will not forgive or forget those responsible for murdering innocent civilians. This statement likely refers to the ongoing conflict between Ukrainian forces and pro-Russian separatists in eastern Ukraine, where numerous civilians have been killed or injured. The President's words suggest a strong stance against any violence or aggression towards their people, and a determination to hold accountable those who commit such heinous acts.
In Irpin, Ukraine, residents were under fire from Russian forces while attempting to flee. Jeremy Bowen was on the frontline, providing live coverage of the situation. The main points are:

1. Location: Irpin, Ukraine
2. Situation: Residents were under fire from Russian forces while trying to flee.
3. Frontline reporter: Jeremy Bowen was on the frontline, providing live coverage of the situation.
The world's largest fertilizer company has warned of potential disruptions to glob

Token indices sequence length is longer than the specified maximum sequence length for this model (1457 > 1024). Running this sequence through the model will result in indexing errors


• In Ukraine, the President has vowed to hold accountable those responsible for killing innocent civilians in the ongoing conflict with pro-Russian separatists.
• Residents of Irpin were under fire from Russian forces while trying to flee, and a frontline reporter was providing live coverage of the situation.
• The world's largest fertilizer company warned of potential disruptions to global food supplies due to the conflict in Ukraine.
• Parents of a young victim of the Manchester Arena bombing spoke about their struggle to cope with the loss of their daughter and the impact on their lives.
• Consumers are facing increased energy costs due to rising fuel prices and higher household bills.
• Boris Johnson will meet with the Prime Ministers of Canada and the Netherlands to discuss various issues, while members of Parliament debate new laws aimed at targeting oligarchs.
• Only about 1% of the 5,500 applications for immigration status under the Windrush scheme have been granted, indicating

In [ ]:
final_mp_data = []
for doc, out in zip(
    map_reduce_outputs["input_documents"], map_reduce_outputs["intermediate_steps"]
):
    output = {}
    output["text"] = doc.page_content
    output["summary"] = out
    final_mp_data.append(output)

In [ ]:
pd.DataFrame.from_dict(final_mp_data)

,text,summary
0,The Ukrainian president says the country will ...,\nIn a statement made by the President of Ukra...
1,"Jeremy Bowen was on the frontline in Irpin, as...","\nIn Irpin, Ukraine, residents were under fire..."
2,One of the world's biggest fertiliser firms sa...,\nThe world's largest fertilizer company has w...
3,The parents of the Manchester Arena bombing's ...,The parents of the youngest victim of the Manc...
4,Consumers are feeling the impact of higher ene...,\nConsumers are facing increased energy costs ...
5,Boris Johnson is to meet the Canadian and Dutc...,"\nBoris Johnson, the Prime Minister of the Uni..."
6,"The home secretary says she is ""surging capaci...",\nThe Home Secretary has announced that only a...
7,TikTok suspends live streaming and new content...,TikTok has suspended live streaming and new co...
8,Five things you need to know about the coronav...,"Sure, here's a summary of the main points and..."
9,"People have been held in 53 cities, from St Pe...","\nIn summary, people have been detained in 53 ..."
